In [12]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.svm import SVR

In [13]:
data1= pd.read_excel("HW_Data_Set.xlsx")
data = data1.copy()
data = data.replace('?',np.nan)
data = data.dropna()
dummy = pd.get_dummies(data[["ind_109"]])
y = data["90_target"]
X_= data.drop(["ind_109","50_target","90_target","20_target", "ind_420", "ind_422"],axis=1).astype('float64')
X = pd.concat([X_,dummy[["ind_109_GREEN"]]],axis=1)
X.head(),

(   ind_5  ind_6  ind_8      ind_9     ind_10     ind_12     ind_13  ind_14  \
 0   19.0   17.0  100.0  85.714286  14.285714  72.363515  60.808814   23.80   
 1   24.0   19.0  100.0  78.571429  21.428571  74.275883  64.366798   11.45   
 2   30.0   24.0  100.0  71.428571  28.571429  75.140402  65.915803    8.75   
 3   37.0   30.0  100.0  64.285714  35.714286  76.677846  68.584234    7.80   
 4   41.0   37.0  100.0  57.142857  42.857143  81.603007  76.455495   14.90   
 
    ind_15  ind_16  ...  ind_407  ind_410   ind_412  ind_414  ind_416  ind_418  \
 0   17.62   11.73  ...    -23.0    -27.0  0.002371      2.0    -49.6    -54.0   
 1   18.16   12.22  ...    -16.0    -31.0  0.003074      0.8    -55.6    -60.0   
 2   17.86   12.28  ...    -10.0    -30.0  0.003561      2.7    -58.4    -60.0   
 3   14.76   12.61  ...    -10.0    -30.0  0.004366      0.9    -61.8    -65.0   
 4   11.92   14.25  ...    -24.0    -45.0  0.004198      0.1    -79.8    -86.0   
 
    ind_424   ind_426   ind_42

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [15]:
svr = SVR().fit(X_train, y_train)

# TAHMİN

In [16]:
y_pred = svr.predict(X_test)

In [17]:
np.sqrt(mean_squared_error(y_test, y_pred))

45.01387120874824

# MODEL TUNING

In [18]:
svr

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [19]:
svr_params = {"C": [0.01, 0.1,0.4,5,10,20,30,40,50]}
svr_cv_model = GridSearchCV(svr,svr_params, cv = 10)
svr_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 0.4, 5, 10, 20, 30, 40, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [20]:
pd.Series(svr_cv_model.best_params_)[0]

50

In [21]:
svr_tuned = SVR("rbf", C = pd.Series(svr_cv_model.best_params_)[0]).fit(X_train, 
                                                                        y_train)


In [22]:
y_pred = svr_tuned.predict(X_test)

np.sqrt(mean_squared_error(y_test, y_pred))

35.72142169624337